# base

In [ ]:
//// test

open testing

## execution

### emit

In [ ]:
inl emit forall t. (x : t) : t =
    $'!x '

### emit_unit

In [ ]:
inl emit_unit forall t. (x : t) : () =
    $'!x '

### use

In [ ]:
inl use forall t. (x : t) : t =
    $'use !x = !x ' : ()
    $'!x '

## target

### backend_switch

In [ ]:
inl backend_switch forall t. x : t =
    real
        !!!!BackendSwitch (
            ("Fsharp", x.Fsharp ()),
            ("Python", x.Python ()),
            ("Cuda", x.Cuda ())
        ) : t

### target_runtime

In [ ]:
union target_runtime =
    | Native
    | Wasm
    | Contract

### target

In [ ]:
union target =
    | Fsharp : target_runtime
    | Cuda : target_runtime
    | Rust : target_runtime
    | TypeScript : target_runtime
    | Python : target_runtime

### run_target

In [ ]:
inl run_target forall t. (fn : target -> (() -> t)) : t =
    backend_switch {
        Fsharp = fun () =>
            inl result = dyn true
            $'let mutable _!result : `t option = None '
            $'\n#if FABLE_COMPILER || WASM || CONTRACT'
            $'\n#if FABLE_COMPILER_RUST && \!WASM && \!CONTRACT'
            fn (Rust Native) () |> emit_unit
            $'#endif\n#if FABLE_COMPILER_RUST && WASM'
            fn (Rust Wasm) () |> emit_unit
            $'#endif\n#if FABLE_COMPILER_RUST && CONTRACT'
            fn (Rust Contract) () |> emit_unit
            $'#endif\n#if FABLE_COMPILER_TYPESCRIPT'
            fn (TypeScript Native) () |> emit_unit
            $'#endif\n#if FABLE_COMPILER_PYTHON'
            fn (Python Native) () |> emit_unit
            $'#endif\n#else'
            fn (Fsharp Native) () |> emit_unit
            $'#endif'
            $'|> fun x -> _!result <- Some x'
            $'match _!result with Some x -> x | None -> failwith "base.run_target / _!result=None"' : t
        Python = fun () =>
            fn (Cuda Native) ()
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

run_target function
    | Fsharp (Native) => fun () => $'1uy'
    | Cuda (Native) => fun () => $'1'
    | Rust (Native) => fun () => $'1uy'
    | TypeScript (Native) => fun () => $'1uy'
    | Python (Native) => fun () => $'1uy'
    | _ => fun () => $'2uy'
|> _assert_eq 1u8

.py output (Cuda):
assert_eq / actual: 1 / expected: 1

.rs output:
assert_eq / actual: 1 / expected: 1

.ts output:
assert_eq / actual: 1 / expected: 1

.py output:
assert_eq / actual: 1 / expected: 1


.fsx output:
assert_eq / actual: 1uy / expected: 1uy


## function

### invoke

In [ ]:
inl invoke fn =
    fn ()

### lazy

In [ ]:
nominal lazy t = $'Lazy<`t>'

### memoize

In [ ]:
nominal lazy t = $'Lazy<`t>'

inl memoize forall t. (fn : () -> t) : () -> t =
    inl fn = join fn
    backend_switch {
        Fsharp = fun () =>
            inl result : lazy t = $'lazy !fn ()'
            fun () => $'!result.Value' : t
        Python = fun () =>
            inl result = mut None
            inl computed = mut false
            fun () =>
                if *computed
                then *result
                else
                    result <- fn () |> Some
                    computed <- true
                    *result
                |> optionm.value
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

inl count = mut 0i32
inl add =
    fun () =>
        count <- *count + 1
        count
    |> memoize

add () |> ignore
add () |> ignore
add () |> ignore

*count
|> _assert_eq 1

.py output (Cuda):
assert_eq / actual: 1 / expected: 1

.rs output:
assert_eq / actual: 1 / expected: 1

.ts output:
assert_eq / actual: 1 / expected: 1

.py output:
assert_eq / actual: 1 / expected: 1


.fsx output:
assert_eq / actual: 1 / expected: 1


### capture

In [ ]:
inl capture forall t. (fn : () -> t) : t =
    inl result = dyn true
    $'let mutable _!result : `t option = None '
    $'('
    $'(fun () ->'
    $'(fun () ->'
    fn () |> emit_unit
    $')'
    $'|> fun x -> x ()'
    $') () )'
    $'|> fun x -> _!result <- Some x'
    $'match _!result with Some x -> x | None -> failwith "base.capture / _!result=None"'

## arithmetic

### (+.)

In [ ]:
inl (+.) forall t. (a : t) (b : t) : t =
    $'!a + !b '

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'3' : i32) +. ($'-6' : i32)
|> _assert_eq -3i32

.py output (Cuda):
assert_eq / actual: -3 / expected: -3

.rs output:
assert_eq / actual: -3 / expected: -3

.ts output:
assert_eq / actual: -3 / expected: -3

.py output:
assert_eq / actual: -3 / expected: -3


.fsx output:
assert_eq / actual: -3 / expected: -3


### (-.)

In [ ]:
inl (-.) forall t. (a : t) (b : t) : t =
    $'!a - !b '

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'3' : i32) -. ($'6' : i32)
|> _assert_eq -3i32

.py output (Cuda):
assert_eq / actual: -3 / expected: -3

.rs output:
assert_eq / actual: -3 / expected: -3

.ts output:
assert_eq / actual: -3 / expected: -3

.py output:
assert_eq / actual: -3 / expected: -3


.fsx output:
assert_eq / actual: -3 / expected: -3


### (*.)

In [ ]:
inl (*.) forall t. (a : t) (b : t) : t =
    $'!a * !b '

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'3' : i32) *. ($'-1' : i32)
|> _assert_eq -3i32

.py output (Cuda):
assert_eq / actual: -3 / expected: -3

.rs output:
assert_eq / actual: -3 / expected: -3

.ts output:
assert_eq / actual: -3 / expected: -3

.py output:
assert_eq / actual: -3 / expected: -3


.fsx output:
assert_eq / actual: -3 / expected: -3


### (/.)

In [ ]:
inl (/.) forall t. (a : t) (b : t) : t =
    $'!a / !b '

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'-3' : i32) /. ($'1' : i32)
|> _assert_eq -3i32

.py output (Cuda):
assert_eq / actual: -3.0 / expected: -3

.rs output:
assert_eq / actual: -3 / expected: -3

.ts output:
assert_eq / actual: -3 / expected: -3

.py output:
assert_eq / actual: -3 / expected: -3


.fsx output:
assert_eq / actual: -3 / expected: -3


## comparison

### (=.)

In [ ]:
inl (=.) forall t. (a : t) (b : t) : bool =
    backend_switch {
        Fsharp = fun () => $'!a = !b ' : bool
        Python = fun () => $'!a == !b ' : bool
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'-3' : i32) =. ($'-3' : i32)
|> _assert_eq true

.py output (Cuda):
assert_eq / actual: True / expected: True

.rs output:
assert_eq / actual: true / expected: true

.ts output:
assert_eq / actual: true / expected: true

.py output:
assert_eq / actual: true / expected: true


.fsx output:
assert_eq / actual: true / expected: true


### (<>.)

In [ ]:
inl (<>.) forall t. (a : t) (b : t) : bool =
    backend_switch {
        Fsharp = fun () => $'!a <> !b ' : bool
        Python = fun () => $'!a \!= !b ' : bool
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'-3' : i32) <>. ($'3' : i32)
|> _assert_eq true

.py output (Cuda):
assert_eq / actual: True / expected: True

.rs output:
assert_eq / actual: true / expected: true

.ts output:
assert_eq / actual: true / expected: true

.py output:
assert_eq / actual: true / expected: true


.fsx output:
assert_eq / actual: true / expected: true


## (<>..)

In [ ]:
inl (<>..) a b =
    fun () => a = b
    |> dyn
    |> invoke
    |> not

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

($'-3' : i32) <>.. ($'3' : i32)
|> _assert_eq true

.py output (Cuda):
assert_eq / actual: True / expected: True

.rs output:
assert_eq / actual: true / expected: true

.ts output:
assert_eq / actual: true / expected: true

.py output:
assert_eq / actual: true / expected: true


.fsx output:
assert_eq / actual: true / expected: true


## composition

### append

In [ ]:
prototype append t : t -> t -> t

### (++)

In [ ]:
inl (++) a b =
    b |> append a

## application

### (||>)

In [ ]:
inl (||>) (arg1, arg2) fn =
    arg2 |> fn arg1

### fix_condition

In [ ]:
inl fix_condition x a b =
    if x ()
    then a () |> fun x => $'(*' : ()
    else
        $'*) else' : ()
        b () |> fun x => $'(*' : ()
    |> fun x => $'*)' : ()

## type

### infer

In [ ]:
nominal infer = $'_'

### any

In [ ]:
nominal any = $'obj'

### unit

In [ ]:
nominal unit = $'unit'

### null

In [ ]:
inl null forall t. () : t =
    backend_switch {
        Fsharp = fun () => $'null |> unbox<`t>' : t
        Python = fun () => $'None' : t
    }

### defaultof

In [ ]:
inl defaultof forall t. () : t =
    $'Unchecked.defaultof<`t>'

### choice2'

In [ ]:
nominal choice2' a b = $'Choice<`a, `b>'

### choice2_unbox

In [ ]:
inl choice2_unbox forall t1 t2. (choice : choice2' t1 t2) : choice2 t1 t2 =
    run_target function
        | Fsharp (Native) => fun () =>
            inl c1of2 (x : t1) : _ _ t2 = C1of2 x
            inl c2of2 (x : t2) : _ t1 _ = C2of2 x
            $'match !choice with Choice1Of2 x -> !c1of2 x | Choice2Of2 x -> !c2of2 x'
        | _ => fun () => null ()

## pair

### pair

In [ ]:
nominal pair a b = $'(`a * `b)'

inl pair x y =
    x, y

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

pair 1i32 2i32
|> _assert_eq (1, 2)

.py output (Cuda):
assert_eq / actual: (1, 2) / expected: (1, 2)

.rs output:
assert_eq / actual: (1, 2) / expected: (1, 2)

.ts output:
assert_eq / actual: 1,2 / expected: 1,2

.py output:
assert_eq / actual: (1, 2) / expected: (1, 2)


.fsx output:
assert_eq / actual: struct (1, 2) / expected: struct (1, 2)


### new_pair

In [ ]:
inl new_pair forall a b. (a : a) (b : b) : pair a b =
    $'!a, !b '

### from_pair

In [ ]:
inl from_pair forall a b. (pair : pair a b) : a * b =
    backend_switch {
        Fsharp = fun () =>
            $'let (a, b) = !pair '
            ($'a' : a), ($'b' : b)
        Python = fun () =>
            $'a, b = !pair '
            ($'a' : a), ($'b' : b)
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

new_pair "a" (new_pair 1i32 "b")
|> from_pair
|> _assert_eq' ("a", (new_pair 1i32 "b"))

.py output (Cuda):
assert_eq' / actual: ('a', (1, 'b')) / expected: ('a', (1, 'b'))

.rs output:
assert_eq' / actual: ("a", (1, "b")) / expected: ("a", (1, "b"))

.ts output:
assert_eq' / actual: a,1,b / expected: a,1,b

.py output:
assert_eq' / actual: ('a', (1, 'b')) / expected: ('a', (1, 'b'))


.fsx output:
assert_eq' / actual: struct ("a", (1, "b")) / expected: struct ("a", (1, "b"))


## ref

### ref

In [ ]:
nominal ref t = $'`t ref'

### new_ref

In [ ]:
inl new_ref forall t. (x : t) : ref t =
    $'ref !x '

### ref_value

In [ ]:
inl ref_value forall t. (x : ref t) : t =
    $'!x.Value'

### ref_set_value

In [ ]:
inl ref_set_value forall t. (value : t) (ref : ref t) : ref t =
    $'!ref.Value <- !value '
    ref

## convert

### convert

In [ ]:
inl convert forall t u. (x : t) : u =
    backend_switch {
        Fsharp = fun () => $'!x |> `u ' : u
        Python = fun () => $'!x ' : u
    }

### unbox

In [ ]:
inl unbox forall t u. (x : t) : u =
    backend_switch {
        Fsharp = fun () => $'!x |> unbox<`u>' : u
        Python = fun () => $'!x ' : u
    }

### u8

In [ ]:
inl u8 forall t. (x : t) : u8 =
    x |> $'uint8'

### u16

In [ ]:
inl u16 forall t. (x : t) : u16 =
    x |> $'uint16'

### u64

In [ ]:
inl u64 forall t. (x : t) : u64 =
    x |> $'uint64'

### i32

In [ ]:
inl i32 forall t. (x : t) : i32 =
    x |> $'int32'

### i64

In [ ]:
inl i64 forall t. (x : t) : i64 =
    x |> $'int64'

### f32

In [ ]:
inl f32 forall t. (x : t) : f32 =
    x |> $'float32'

### f64

In [ ]:
inl f64 forall t. (x : t) : f64 =
    x |> $'float'

### unativeint

In [ ]:
nominal unativeint = $'unativeint'

### convert_i32

In [ ]:
inl convert_i32 forall t. (x : t) : i32 =
    x |> $'System.Convert.ToInt32'

### convert_i32_base

In [ ]:
inl convert_i32_base forall t. (base : i32) (x : t) : i32 =
    $'System.Convert.ToInt32 (!x, !base)'

## error

### exn

In [ ]:
nominal exn = $"backend_switch `({ Fsharp : $'exn'; Python : $'BaseException' })"

inl exn x =
    x |> $'`exn '

### try

In [ ]:
inl try forall t. (fn : () -> t) (ex_fn : exn -> option t) : option t =
    backend_switch {
        Fsharp = fun () =>
            inl some x : option t = Some x
            inl some = dyn some
            inl fn = dyn fn
            inl ex_fn = dyn ex_fn
            $'let result = ref !(None : option t)'
            $'try'
            $'    result.Value <- !fn () |> !some '
            $'with ex ->'
            $'    result.Value <- !ex_fn ex '
            $'result.Value' : option t
        Python = fun () =>
            $'result = !(None : option t)'
            inl fn = dyn fn
            inl ex_fn = dyn ex_fn
            $'try:'
            $'    result = !fn()\n        \'\'\''
            $'\'\'\''
            $'except Exception as e:'
            $'    result = !ex_fn(e)'
            $'result' : option t
    }

In [ ]:
//// test
///! fsharp
////! cuda // cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version
///! rust
///! typescript
///! python

try
    fun () => a ;[ 0i32 ] |> am'.index 1i32 |> sm'.format
    (fun ex => $'!ex ' |> sm'.format_exception |> Some)
|> optionm.value
|> _assert_eq (run_target function
    | Fsharp => fun () => "System.IndexOutOfRangeException: Index was outside the bounds of the array."
    | Cuda => fun () => "array index out of range"
    | Rust => fun () => "Exception { message: \"index out of bounds: the len is 1 but the index is 1\" }"
    | TypeScript => fun () => "Error: Index was outside the bounds of the array.\\nParameter name: index"
    | Python => fun () => "array index out of range"
)

.rs output:
assert_eq / actual: "Exception { message: "index out of bounds: the len is 1 but the index is 1" }" / expected: "Exception { message: "index out of bounds: the len is 1 but the index is 1" }"

.ts output:
assert_eq / actual: Error: Index was outside the bounds of the array.\nParameter name: index / expected: Error: Index was outside the bounds of the array.\nParameter name: index

.py output:
assert_eq / actual: array index out of range / expected: array index out of range


.fsx output:
assert_eq / actual: "System.IndexOutOfRangeException: Index was outside the bounds of the array." / expected: "System.IndexOutOfRangeException: Index was outside the bounds of the array."


### try_unit

In [ ]:
inl try_unit forall t. (fn : () -> ()) (ex_fn : exn -> ()) : t =
    $'try'
    fn ()
    |> ignore
    $'with ex ->'
    ex_fn $'ex'
    |> ignore
    $'(*'
    $'*)'

### try_finally

In [ ]:
inl try_finally forall t. (fn : () -> ()) (finally : () -> ()) : t =
    $'try'
    fn ()
    |> ignore
    $'finally'
    finally ()
    |> ignore
    $'(*'
    $'*)'